# V2 de RICK

In [19]:
import os
import numpy as np
import shutil
import matplotlib.pyplot as plt
from vozyaudio import lee_audio, espectro as calcular_espectro
import subprocess
from scipy.signal import firwin, lfilter, get_window
from scipy.fft import rfft, rfftfreq


In [42]:
AUDIO_PATH = "audios\music.wav"
CARPETA_FRAMES  = 'fotogramas'
FPS = 60
COLOR = "color" # gris, color
MODE = "ampl"    # ampl, fir, fft
# Eliminar y crear carpeta de fotogramas
if os.path.exists(CARPETA_FRAMES):
    shutil.rmtree(CARPETA_FRAMES)
os.makedirs(CARPETA_FRAMES)

In [21]:
# Leer audio
fs, audio = lee_audio(AUDIO_PATH)
audio = audio / np.max(np.abs(audio))  # Normalización

In [23]:
# Filtros predefinidos (definidos fuera para no recalcular en cada llamada)
numcoef = 101
b_low = firwin(numcoef, cutoff=2000, fs=fs)
b_band = firwin(numcoef, [2000, 6000], pass_zero=False, fs=fs)
b_high = firwin(numcoef, cutoff=6000, fs=fs, pass_zero=False)

# Codificador

In [50]:
def generar_frame_cabecera(N, salto, fps, carpeta=CARPETA_FRAMES):
    """
    Genera el primer frame (frame_0000.png) con los metadatos necesarios
    para la decodificación de audio desde vídeo.

    Parameters
    ----------
    N : int
        Tamaño de la ventana (número de columnas y filas del frame cuadrado).
    salto : int
        Paso (hop size) entre ventanas en muestras.
    fps : int
        Número de frames por segundo usado al crear el vídeo.
    carpeta : str, optional
        Ruta de la carpeta donde se guardan los frames (por defecto CARPETA_FRAMES).
    """
    # Construir matriz de ceros
    header = np.zeros((N, N), dtype=np.uint8)

    # Codificar N (2 bytes), salto (2 bytes) y fps (1 byte)
    header[0, 0] = (N // 256) & 0xFF
    header[0, 1] = N % 256
    header[1, 0] = (salto // 256) & 0xFF
    header[1, 1] = salto % 256
    header[2, 0] = fps & 0xFF

    # Guardar como imagen en la carpeta indicada
    ruta = os.path.join(carpeta, "frame_0000.png")
    plt.imsave(ruta, header, cmap='gray')


In [ ]:
def generarFrame_AudioGeneral():
    """
    Cada fila del frame es una ventana distinta de audio.
    El frame final será de tamaño (n_bloques, N)
    """
    frame_data = []

    for i in range(n_bloques):
        print(f"\rProcesando bloque {i+1}/{n_bloques}", end='', flush=True)

        # Extraer y enventanar bloque
        start = i * salto
        bloque = audio[start : start + N] * ventana

        # Normalizar bloque individualmente a [0, 255]
        bloque_norm = (bloque - np.min(bloque)) / (np.max(bloque) - np.min(bloque) + 1e-12)
        fila = (bloque_norm * 255).astype(np.uint8)

        frame_data.append(fila)

    # Convertir a imagen: cada fila es un bloque
    frame_img = np.vstack(frame_data)

    # Guardar una única imagen
    plt.imsave(f"{CARPETA_FRAMES}/frame_total.png", frame_img, cmap='gray')
    print("\n✅ Frame generado.")


In [25]:
def colorearFila(bloque, modo='amplitud'):
    """
    Devuelve una fila RGB (shape 1×N×3, uint8) coloreada según:
      - modo='amplitud': mapa basado en amplitud temporal
      - modo='fft'      : mapa basado en energía por bin de la FFT
    """
    # Normalización ventana
    bloque = bloque.astype(np.float32)
    if modo == 'ampl':
        amp = np.abs(bloque)
        amp = (amp - amp.min()) / (amp.max() - amp.min() + 1e-12)
        r = 1 - amp
        g = amp
        b = 0.5 * np.ones_like(amp)

    elif modo == 'fft':
        # FFT
        X = np.abs(rfft(bloque, n=N))
        f = rfftfreq(N, 1/fs)
        Xn = (X - X.min())/(X.max() - X.min() + 1e-12)

        # mascarillas
        low_mask  = (f <= 500)
        band_mask = (f > 500) & (f < 5000)
        high_mask = (f >= 5000)

        Rf = Xn * low_mask
        Bf = Xn * band_mask
        Gf = Xn * high_mask

        # interpolación a N columnas
        bins   = np.arange(len(Xn))
        target = np.linspace(0, len(Xn)-1, N)
        interp = lambda arr: np.interp(target, bins, arr)

        r = interp(Rf)
        g = interp(Gf)
        b = interp(Bf)
    elif modo == 'fir':
        # filtrar en el dominio temporal y colorear por muestra
        y_low  = lfilter(b_low,  1.0, bloque)
        y_band = lfilter(b_band, 1.0, bloque)
        y_high = lfilter(b_high, 1.0, bloque)

        E_l = np.abs(y_low)
        E_b = np.abs(y_band)
        E_h = np.abs(y_high)
        # normaliza cada uno 0..1
        r = (E_l - E_l.min())/(E_l.max()-E_l.min()+1e-12)
        g = (E_h - E_h.min())/(E_h.max()-E_h.min()+1e-12)
        b = (E_b - E_b.min())/(E_b.max()-E_b.min()+1e-12)

    else:
        raise ValueError("modo debe ser 'amplitud', 'fft' o 'fir'")

    # stack y convertir a uint8
    fila = np.stack([r, g, b], axis=1)[np.newaxis, ...]  # (1×N×3)
    return (fila * 255).astype(np.uint8)


In [51]:
def generarFrames_VentanaFrame(color="gris", modo="amplitud", window_type="hann",salto = None ):
    """
    Genera frames donde cada fila del frame representa un bloque de audio.
    
    Parámetros:
    - color: "gris" o "color"
    - modo: para el coloreado, "amplitud", "fft" o "fir"
    - window_type: nombre de la ventana (p. ej. "hann", "hamming", "boxcar", "blackman", etc.)
    
    """

    # 1) calculamos N a partir de FPS
    N = fs // FPS
    if N % 2 != 0:
        N += 1

    # 2) elegimos el salto
    if salto is None:
        salto = N // 2

    print(f"Tamaño de ventana: {N}, salto: {salto}, resolución: {N}x{N}")

    # 3) ventana de análisis
    ventana = get_window(window_type, N)

    # 4) número total de bloques
    n_bloques = (len(audio) - N) // salto
    print(f"Numero de bloques {n_bloques}")

    # Frame 0 - metadatos
    generar_frame_cabecera(N, salto, FPS, CARPETA_FRAMES)

    for i in range(n_bloques):
        print(f"\rProcesando bloque {i+1}/{n_bloques}", end='', flush=True)

        start = i * salto
        bloque = audio[start : start + N] * ventana

        if color == "gris":
            bloque_norm = (bloque - np.min(bloque)) / (np.max(bloque) - np.min(bloque) + 1e-12)
            fila = (bloque_norm * 255).astype(np.uint8)
            frame_img = np.tile(fila, (N, 1))
            plt.imsave(f"{CARPETA_FRAMES}/frame_{i + 1:04d}.png", frame_img, cmap='gray')

        elif color == "color":
            fila = colorearFila(bloque, modo)  # ampl , fft , FIR
            frame_img = np.tile(fila, (N, 1, 1))
            plt.imsave(f"{CARPETA_FRAMES}/frame_{i+1:04d}.png", frame_img)

        else:
            raise ValueError("Modo no reconocido. Usa 'gris' o 'color'.")


In [52]:
generarFrames_VentanaFrame(COLOR,MODE)

Tamaño de ventana: 736, salto: 368, resolución: 736x736
Numero de bloques 1196
Procesando bloque 1196/1196

In [44]:
# 1) Carpeta export y rutas
os.makedirs("exports", exist_ok=True)
audio_base   = os.path.splitext(os.path.basename(AUDIO_PATH))[0]
output_name  = f"{audio_base}_{MODE}_{COLOR}.mp4"
output_path  = os.path.join("exports", output_name)

# 2) Borra antiguo
if os.path.exists(output_path):
    os.remove(output_path)

# 3) Ejecuta el batch desde Jupyter, expandiendo las variables
!generarVideo2.bat {FPS} "{output_path}"


c:\Users\luigi\Documents\UPV\3ro\Voz\PixelSounds>ffmpeg -framerate 60 -i fotogramas/frame_%04d.png -c:v libx264 -pix_fmt yuv420p exports\music_ampl_color.mp4 


ffmpeg version 7.1-essentials_build-www.gyan.dev Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 14.2.0 (Rev1, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-zlib --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-sdl2 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-mediafoundation --enable-libass --enable-libfreetype --enable-libfribidi --enable-libharfbuzz --enable-libvidstab --enable-libvmaf --enable-libzimg --enable-amf --enable-cuda-llvm --enable-cuvid --enable-dxva2 --enable-d3d11va --enable-d3d12va --enable-ffnvcodec --enable-libvpl --enable-nvdec --enable-nvenc --enable-vaapi --enable-libgme --enable-libopenmpt --enable-libopencore-amrwb --enable-libmp3lame --e

In [45]:
if os.path.exists(CARPETA_FRAMES):
    shutil.rmtree(CARPETA_FRAMES)
os.makedirs(CARPETA_FRAMES)

# Decodificador